In [3]:
'''
Tyler Sledge
CSC 485
Sample Recurrent Neural Network Architecture
'''

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          64000     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


In [5]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          64000     
_________________________________________________________________
gru (GRU)                    (None, None, 256)         247296    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 361,866
Trainable params: 361,866
Non-trainable params: 0
_________________________________________________________________


In [6]:
encoder_vocab = 1000
decoder_vocab = 2000

encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(
    encoder_input
)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(
    encoder_embedded
)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(
    decoder_input
)

# Pass the 2 states to a new LSTM layer, as initial state
decoder_output = layers.LSTM(64, name="decoder")(
    decoder_embedded, initial_state=encoder_state
)
output = layers.Dense(10)(decoder_output)

model = keras.Model([encoder_input, decoder_input], output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     64000       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     128000      input_2[0][0]                    
______________________________________________________________________________________________

In [7]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)
output = lstm_layer(paragraph3)

# reset_states() will reset the cached state to the original initial_state.
# If no initial_state was provided, zero-states will be used by default.
lstm_layer.reset_states()

In [8]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 28

units = 64
output_size = 10  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model

## Load dataset

In [9]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

11493376/11490434 [==============================] - 0s 0us/step


### CuDNN Model

In [10]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=1
)

938/938 [==============================] - 20s 19ms/step - loss: 1.3214 - accuracy: 0.5718 - val_loss: 1.5953 - val_accuracy: 0.4630


### Non CuDNN Model

In [11]:
noncudnn_model = build_model(allow_cudnn_kernel=False)
noncudnn_model.set_weights(model.get_weights())
noncudnn_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)
noncudnn_model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=1
)

938/938 [==============================] - 19s 18ms/step - loss: 0.4427 - accuracy: 0.8683 - val_loss: 0.3946 - val_accuracy: 0.8643


## Second Model


In [12]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=20
)

Epoch 1/20
938/938 [==============================] - 19s 19ms/step - loss: 1.3186 - accuracy: 0.5686 - val_loss: 0.5196 - val_accuracy: 0.8359
Epoch 2/20
938/938 [==============================] - 17s 19ms/step - loss: 0.4559 - accuracy: 0.8581 - val_loss: 0.3468 - val_accuracy: 0.8888
Epoch 3/20
938/938 [==============================] - 17s 19ms/step - loss: 0.2757 - accuracy: 0.9186 - val_loss: 0.2185 - val_accuracy: 0.9373
Epoch 4/20
938/938 [==============================] - 17s 19ms/step - loss: 0.2140 - accuracy: 0.9356 - val_loss: 0.1640 - val_accuracy: 0.9483
Epoch 5/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1728 - accuracy: 0.9485 - val_loss: 0.1643 - val_accuracy: 0.9504
Epoch 6/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1554 - accuracy: 0.9540 - val_loss: 0.2141 - val_accuracy: 0.9283
Epoch 7/20
938/938 [==============================] - 17s 18ms/step - loss: 0.1385 - accuracy: 0.9584 - val_loss: 0.1194 - val_accuracy:

In [13]:
## Compare with nonCuDNN

noncudnn_model = build_model(allow_cudnn_kernel=False)
noncudnn_model.set_weights(model.get_weights())
noncudnn_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)
noncudnn_model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=20
)

Epoch 1/20
938/938 [==============================] - 18s 18ms/step - loss: 0.0591 - accuracy: 0.9818 - val_loss: 0.0638 - val_accuracy: 0.9796
Epoch 2/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0605 - accuracy: 0.9814 - val_loss: 0.0722 - val_accuracy: 0.9766
Epoch 3/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0585 - accuracy: 0.9820 - val_loss: 0.0600 - val_accuracy: 0.9809
Epoch 4/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0541 - accuracy: 0.9843 - val_loss: 0.1403 - val_accuracy: 0.9588
Epoch 5/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0566 - accuracy: 0.9824 - val_loss: 0.0662 - val_accuracy: 0.9788
Epoch 6/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0531 - accuracy: 0.9838 - val_loss: 0.0591 - val_accuracy: 0.9804
Epoch 7/20
938/938 [==============================] - 17s 18ms/step - loss: 0.0503 - accuracy: 0.9834 - val_loss: 0.0771 - val_accuracy: